## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

In [2]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, random_state=42, test_size=0.2)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

tree_clf = DecisionTreeClassifier()
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_leaf_nodes':   2**np.arange(1, 10),
    'min_samples_leaf': range(1, 50, 2)
}

grid_search_tree = GridSearchCV(tree_clf, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search_tree.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_leaf_nodes': array([  2,   4,   8,  16,  32,  64, 128, 256, 512], dtype=int32),
                         'min_samples_leaf': range(1, 50, 2)},
             scoring='accuracy')

In [6]:
import pandas as pd

pd.DataFrame(grid_search_tree.cv_results_).sort_values(by="mean_test_score", ascending=False).head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_leaf_nodes,param_min_samples_leaf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
329,0.021632,0.004813,0.000000,0.000000,entropy,32,9,"{'criterion': 'entropy', 'max_leaf_nodes': 32,...",0.863142,0.871766,0.858215,0.864374,0.005601,1
328,0.016998,0.005740,0.002914,0.004121,entropy,32,7,"{'criterion': 'entropy', 'max_leaf_nodes': 32,...",0.862767,0.871766,0.858215,0.864249,0.005631,2
344,0.018083,0.001566,0.001858,0.002628,entropy,32,39,"{'criterion': 'entropy', 'max_leaf_nodes': 32,...",0.862767,0.871766,0.857839,0.864124,0.005766,3
342,0.012090,0.002762,0.006462,0.004570,entropy,32,35,"{'criterion': 'entropy', 'max_leaf_nodes': 32,...",0.862767,0.871766,0.857839,0.864124,0.005766,3
343,0.018185,0.003835,0.000000,0.000000,entropy,32,37,"{'criterion': 'entropy', 'max_leaf_nodes': 32,...",0.862767,0.871766,0.857839,0.864124,0.005766,3


In [7]:
from sklearn.metrics import accuracy_score
best_tree = grid_search_tree.best_estimator_
accuracy_score(y_test, best_tree.predict(X_test))


0.8675

2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [16]:
from sklearn.model_selection import ShuffleSplit

m = X_train.shape[0]

forests = []
shuffleSplit = ShuffleSplit(n_splits=1500, train_size=600 / m, random_state=42)
best_parmas = grid_search_tree.best_params_

for train_index, test_index in shuffleSplit.split(X_train):
    tree_clf_sample = DecisionTreeClassifier(**best_parmas)
    tree_clf_sample.fit(X_train[train_index], y_train[train_index])
    forests.append(tree_clf_sample)

In [17]:
scores = []
predictions = []
for tree in forests:
    pred = tree.predict(X_test)
    predictions.append(pred)
    scores.append(accuracy_score(y_test, tree.predict(X_test)))


In [18]:
np.max(scores)
np.min(scores)
np.mean(scores)

# predictions

0.842738

In [19]:
# 按列 挑选最频繁的结果， 最终选择最频繁出现的结果
from scipy import stats
forest_pred = stats.mode(predictions, axis=0).mode
forest_pred.shape
accuracy_score(y_test, forest_pred)

0.862